In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
import numpy as np

# Load the data
info_A1 = pd.read_csv(r"C:\Users\pingk\Downloads\fadhli nitip\asik.csv")

In [2]:
df = info_A1

# Function to remove outliers
def remove_outliers(df):
    # Select numeric columns
    numeric_cols = df.select_dtypes(include=['number']).columns
    df_numeric = df[numeric_cols]
    
    iso = IsolationForest(contamination=0.05)
    yhat = iso.fit_predict(df_numeric)
    mask = yhat != -1
    df_cleaned = df[mask].reset_index(drop=True)
    return df_cleaned

df = remove_outliers(df)

# Standardize the numeric data
numeric_cols = df.select_dtypes(include=['number']).columns
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Identify numeric and string columns
string_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

# Check the first few rows of the prepared data
print(df.head())

  prov_char country_char               wavenumber   649.893   650.376  \
0       BBX           ID  ID-BBX-068-2307-031_001  0.384336  0.623262   
1       BBX           ID  ID-BBX-068-2307-031_002 -0.334257 -0.516665   
2       BBX           ID  ID-BBX-068-2307-031_003  0.343061  0.382855   
3       BBX           ID  ID-BBX-068-2307-031_004 -0.455392 -0.648278   
4       BBX           ID  ID-BBX-068-2307-031_005 -0.233804 -0.426464   

    650.858    651.34   651.822   652.304   652.786  ...  3998.194  3998.676  \
0  0.790731  0.849385  0.765928  0.530749  0.169306  ... -0.124608 -0.124839   
1 -0.666844 -0.774808 -0.833336 -0.839365 -0.799079  ...  0.001292 -0.013375   
2  0.346922  0.218928 -0.002596 -0.294666 -0.617047  ... -0.213015 -0.294168   
3 -0.920590 -1.227561 -1.512727 -1.726495 -1.851537  ...  0.377603  0.361953   
4 -0.599842 -0.754752 -0.898472 -1.037614 -1.173077  ...  0.530364  0.514048   

   3999.158   3999.64  4000.122  tgp_name  dgp_name  fgp_name  country_name  \
0

In [3]:
numeric_cols

Index(['649.893', '650.376', '650.858', '651.34', '651.822', '652.304',
       '652.786', '653.268', '653.75', '654.232',
       ...
       '3995.783', '3996.265', '3996.747', '3997.23', '3997.712', '3998.194',
       '3998.676', '3999.158', '3999.64', '4000.122'],
      dtype='object', length=6950)

In [4]:
string_cols

['prov_char',
 'country_char',
 'wavenumber',
 'tgp_name',
 'dgp_name',
 'fgp_name',
 'country_name',
 'thnoth_name']

In [5]:
from sklearn.preprocessing import LabelEncoder

# Encode the target variable
label_encoder = LabelEncoder()
df['country_name_encoded'] = label_encoder.fit_transform(df['country_name'])

# Check the encoded target variable
print(df[['country_name', 'country_name_encoded']])

    country_name  country_name_encoded
0      Indonesia                     0
1      Indonesia                     0
2      Indonesia                     0
3      Indonesia                     0
4      Indonesia                     0
..           ...                   ...
410     Thailand                     2
411     Thailand                     2
412     Thailand                     2
413     Thailand                     2
414     Thailand                     2

[415 rows x 2 columns]


C:\Users\pingk\AppData\Local\Temp\ipykernel_1628\2480830658.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['country_name_encoded'] = label_encoder.fit_transform(df['country_name'])


In [7]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

# Define the Random Forest model
model_rf = RandomForestClassifier(random_state=42)

# Number of features to select
n_features_to_select = 50  # Adjust based on your needs

# Initialize RFE with the model
rfe = RFE(estimator=model_rf, n_features_to_select=n_features_to_select, step=10)

# Prepare features and target
X = df.drop(columns=['country_name', 'country_name_encoded', 'prov_char', 'country_char', 'wavenumber', 'thnoth_name', 'tgp_name', 'dgp_name', 'fgp_name'])
y = df['country_name_encoded']

# Fit RFE on the dataset
rfe.fit(X, y)

# Identify the selected features
selected_features = X.columns[rfe.support_]
print("Selected Features:")
print(selected_features)


Selected Features:
Index(['664.839', '665.321', '665.803', '807.546', '1053.907', '1054.389',
       '1054.871', '1055.836', '1061.139', '1062.103', '1062.585', '1063.067',
       '1138.76', '1140.688', '1159.491', '1159.973', '1160.455', '1166.24',
       '1167.205', '1168.169', '1168.651', '1169.133', '1175.883', '1187.936',
       '1188.418', '1188.9', '1467.081', '1491.187', '1539.399', '1539.881',
       '1666.195', '1668.124', '1677.284', '1677.766', '1678.248', '1678.73',
       '1717.3', '1718.264', '1738.031', '1752.494', '1925.574', '2225.933',
       '2226.415', '2226.897', '2895.111', '2895.593', '2916.806', '2917.77',
       '2918.735', '2919.217'],
      dtype='object')


In [8]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Define the SVM model
model_svm = SVC(random_state=42)

# Parameter grid for GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

# Initialize RFE with SVM
rfe_svm = RFE(estimator=model_svm, n_features_to_select=n_features_to_select, step=10)
X_rfe_svm = rfe_svm.fit_transform(X, y)

# Set up GridSearchCV
grid_search_svm = GridSearchCV(estimator=model_svm, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Fit the model with the RFE-selected features
grid_search_svm.fit(X_rfe_svm, y)

# Get the best parameters and score
print("Best parameters:", grid_search_svm.best_params_)
print("Best cross-validation score:", grid_search_svm.best_score_)


ValueError: when `importance_getter=='auto'`, the underlying estimator SVC should have `coef_` or `feature_importances_` attribute. Either pass a fitted estimator to feature selector or call fit before calling transform.

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

# Define the best SVM model with the obtained parameters
best_svm = SVC(**grid_search_svm.best_params_, random_state=42)

# Perform 10-fold cross-validation with the tuned SVM model
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Initialize lists to store results
train_accuracies = []
test_accuracies = []
y_true_all = []
y_pred_all = []

# Apply 10-fold cross-validation
for train_index, test_index in kf.split(X_rfe_svm):
    X_train, X_test = X_rfe_svm[train_index], X_rfe_svm[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the model
    best_svm.fit(X_train, y_train)
    
    # Predict the train and test samples
    y_train_pred = best_svm.predict(X_train)
    y_test_pred = best_svm.predict(X_test)
    
    # Calculate and store train and test accuracies
    train_accuracies.append(accuracy_score(y_train, y_train_pred))
    test_accuracies.append(accuracy_score(y_test, y_test_pred))
    
    # Store the prediction and actual value
    y_true_all.extend(y_test)
    y_pred_all.extend(y_test_pred)

# Calculate average accuracies
avg_train_accuracy = np.mean(train_accuracies)
avg_test_accuracy = np.mean(test_accuracies)

print(f'10-Fold CV with Tuned SVM - Average Train Accuracy: {avg_train_accuracy}')
print(f'10-Fold CV with Tuned SVM - Average Test Accuracy: {avg_test_accuracy}')

# Generate the classification report for the overall test predictions
report_svm_tuned = classification_report(y_true_all, y_pred_all, target_names=label_encoder.classes_, output_dict=True)
print("10-Fold CV Tuned SVM Model Classification Report")
print(classification_report(y_true_all, y_pred_all, target_names=label_encoder.classes_))

# Plot the confusion matrix
cm_tuned_svm = confusion_matrix(y_true_all, y_pred_all)
disp_tuned_svm = ConfusionMatrixDisplay(confusion_matrix=cm_tuned_svm, display_labels=label_encoder.classes_)
disp_tuned_svm.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix: 10-Fold CV Tuned SVM')
plt.show()


In [ ]:
# Define the best SVM model with the obtained parameters
best_svm = SVC(**grid_search_svm.best_params_, random_state=42)

# Initialize lists to store results
train_accuracies = []
test_accuracies = []
y_true_all = []
y_pred_all = []

# Apply 40-fold cross-validation
kf = KFold(n_splits=40, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X_rfe_svm):
    X_train, X_test = X_rfe_svm[train_index], X_rfe_svm[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the model
    best_svm.fit(X_train, y_train)
    
    # Predict the train and test samples
    y_train_pred = best_svm.predict(X_train)
    y_test_pred = best_svm.predict(X_test)
    
    # Calculate and store train and test accuracies
    train_accuracies.append(accuracy_score(y_train, y_train_pred))
    test_accuracies.append(accuracy_score(y_test, y_test_pred))
    
    # Store the prediction and actual value
    y_true_all.extend(y_test)
    y_pred_all.extend(y_test_pred)

# Calculate average accuracies
avg_train_accuracy = np.mean(train_accuracies)
avg_test_accuracy = np.mean(test_accuracies)

print(f'40-Fold CV with Tuned SVM - Average Train Accuracy: {avg_train_accuracy}')
print(f'40-Fold CV with Tuned SVM - Average Test Accuracy: {avg_test_accuracy}')

# Generate the classification report for the overall test predictions
report_svm_tuned = classification_report(y_true_all, y_pred_all, target_names=label_encoder.classes_, output_dict=True)
print("40-Fold CV Tuned SVM Model Classification Report")
print(classification_report(y_true_all, y_pred_all, target_names=label_encoder.classes_))

# Plot the confusion matrix
cm_tuned_svm = confusion_matrix(y_true_all, y_pred_all)
disp_tuned_svm = ConfusionMatrixDisplay(confusion_matrix=cm_tuned_svm, display_labels=label_encoder.classes_)
disp_tuned_svm.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix: 40-Fold CV Tuned SVM')
plt.show()


In [ ]:
from sklearn.model_selection import LeaveOneGroupOut

# Define the best SVM model with the obtained parameters
best_svm = SVC(**grid_search_svm.best_params_, random_state=42)

# Initialize Leave-One-Group-Out Cross-Validation
logo = LeaveOneGroupOut()

# List of provinces to use as groups
prov_codes = ['SKM', 'PLG', 'JHR', 'SSX', 'PNA', 'KTX']

# Map 'prov_char' to integer labels
groups = df['prov_char']

# Filter out samples not in the selected provinces
X_filtered = X_rfe_svm[groups != -1]
y_filtered = y[groups != -1]
groups_filtered = [g for g in groups if g != -1]

# Initialize lists to store results
train_accuracies = []
test_accuracies = []
y_true_all = []
y_pred_all = []

# Apply LOGO-CV
for train_index, test_index in logo.split(X_filtered, y_filtered, groups_filtered):
    X_train, X_test = X_filtered[train_index], X_filtered[test_index]
    y_train, y_test = y_filtered[train_index], y_filtered[test_index]
    
    # Fit the model
    best_svm.fit(X_train, y_train)
    
    # Predict the train and test samples
    y_train_pred = best_svm.predict(X_train)
    y_test_pred = best_svm.predict(X_test)
    
    # Calculate and store train and test accuracies
    train_accuracies.append(accuracy_score(y_train, y_train_pred))
    test_accuracies.append(accuracy_score(y_test, y_test_pred))
    
    # Store the prediction and actual value
    y_true_all.extend(y_test)
    y_pred_all.extend(y_test_pred)

# Calculate average accuracies
avg_train_accuracy = np.mean(train_accuracies)
avg_test_accuracy = np.mean(test_accuracies)

print(f'Leave-One-Province-Out CV with Tuned SVM - Average Train Accuracy: {avg_train_accuracy}')
print(f'Leave-One-Province-Out CV with Tuned SVM - Average Test Accuracy: {avg_test_accuracy}')

# Generate the classification report for the overall test predictions
report_svm_logo = classification_report(y_true_all, y_pred_all, target_names=label_encoder.classes_, output_dict=True)
print("Leave-One-Province-Out CV Tuned SVM Model Classification Report")
print(classification_report(y_true_all, y_pred_all, target_names=label_encoder.classes_))

# Plot the confusion matrix
cm_logo_svm = confusion_matrix(y_true_all, y_pred_all)
disp_logo_svm = ConfusionMatrixDisplay(confusion_matrix=cm_logo_svm, display_labels=label_encoder.classes_)
disp_logo_svm.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix: Leave-One-Province-Out CV Tuned SVM')
plt.show()
